In [3]:
from jira import JIRA

jira = JIRA('https://jira.sonarsource.com/')

In [21]:
issues_in_proj = jira.search_issues('project=SONAR & type=Bug')

In [25]:
size = 100
initial = 0
while True:
    start= initial*size
    issues = jira.search_issues('project=SONAR',  start,size)
    if len(issues) == 0:
        break
    initial += 1
    for issue in issues:
        print('ticket-no=',issue)
        print('IssueType=',issue.fields.issuetype.name)
        print('Status=',issue.fields.status.name)
        print('Versions=',issue.fields.versions)
        print('Summary=',issue.fields.summary)

ticket-no= SONAR-13912
IssueType= Task
Status= Open
Versions= []
Summary= Add OWASP Dependency Check task to Cirrus CI
ticket-no= SONAR-13910
IssueType= Improvement
Status= Open
Versions= []
Summary= Upgrade CFamily analyzer
ticket-no= SONAR-13909
IssueType= New Feature
Status= In Review
Versions= []
Summary= C# and VB.NET analysis updates
ticket-no= SONAR-13908
IssueType= Improvement
Status= In Review
Versions= []
Summary= Upgrade VBnet analyzer
ticket-no= SONAR-13907
IssueType= Improvement
Status= In Review
Versions= []
Summary= Upgrade Javascript analyzer
ticket-no= SONAR-13906
IssueType= Improvement
Status= In Progress
Versions= []
Summary= Prevent ALM binding configuration modal from closing when clicking outside
ticket-no= SONAR-13905
IssueType= Task
Status= In Progress
Versions= []
Summary= Fix potential vulnerabilities of bundled libraries
ticket-no= SONAR-13903
IssueType= Task
Status= In Progress
Versions= []
Summary= Remove use of organizations in Quality Profiles and Active 

ticket-no= SONAR-13766
IssueType= Bug
Status= Closed
Versions= []
Summary= PostProjectAnalysisTask's  'description' is a breaking change
ticket-no= SONAR-13765
IssueType= Improvement
Status= Open
Versions= [<JIRA Version: name='8.4.1', id='16358'>]
Summary= Permission page doesn't behave properly at minimum width
ticket-no= SONAR-13764
IssueType= Improvement
Status= Open
Versions= [<JIRA Version: name='8.4.1', id='16358'>]
Summary= Do not display two error messages simultaneously on issue page
ticket-no= SONAR-13763
IssueType= New Feature
Status= Open
Versions= []
Summary= VB.NET analyzer updates
ticket-no= SONAR-13762
IssueType= Improvement
Status= Open
Versions= []
Summary= Gitlab project onboarding is not align properly
ticket-no= SONAR-13761
IssueType= Improvement
Status= Open
Versions= []
Summary= Unable to access project information/settings from a PR
ticket-no= SONAR-13760
IssueType= New Feature
Status= Open
Versions= []
Summary= Online licence activation
ticket-no= SONAR-13759


ticket-no= SONAR-13594
IssueType= Bug
Status= Closed
Versions= [<JIRA Version: name='8.4', id='15833'>]
Summary= Upgrade to 8.4 fails on MS SQL Server
ticket-no= SONAR-13592
IssueType= Bug
Status= Closed
Versions= []
Summary= File facets on issues page are broken if using the facet search
ticket-no= SONAR-13591
IssueType= Bug
Status= Closed
Versions= [<JIRA Version: name='8.3', id='15640'>]
Summary= Rule tags are no more copied to issues tags
ticket-no= SONAR-13590
IssueType= Bug
Status= Closed
Versions= [<JIRA Version: name='8.4', id='15833'>]
Summary= SonarQube is not starting when in need of issues indexing
ticket-no= SONAR-13589
IssueType= Bug
Status= Closed
Versions= [<JIRA Version: name='8.4', id='15833'>]
Summary= Creation menu tries to connect to Developer Edition+ only WS on Community Edition
ticket-no= SONAR-13588
IssueType= Bug
Status= Open
Versions= []
Summary= Tags defined on external rules are not propagated to external issues
ticket-no= SONAR-13587
IssueType= Improvement

KeyboardInterrupt: 

In [20]:
import pprint
pp = pprint.PrettyPrinter(depth=6)
pp.pprint(issues_in_proj[0].fields.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'PropertyHolder' objects>,
              '__doc__': None,
              '__module__': 'jira.resources',
              '__weakref__': <attribute '__weakref__' of 'PropertyHolder' objects>,
              'assignee': <JIRA User: displayName='Wouter Admiraal', key='wouter.admiraal', name='wouter.admiraal'>,
              'components': [<JIRA Component: name='Branch & PR', id='11094'>,
                             <JIRA Component: name='Web', id='11366'>],
              'created': '2020-09-23T11:30:41.000+0200',
              'creator': <JIRA User: displayName='Pierre Guillot', key='pierre.guillot', name='pierre.guillot'>,
              'customfield_10235': None,
              'customfield_10243': None,
              'customfield_10431': None,
              'customfield_10432': None,
              'customfield_10433': None,
              'customfield_10434': None,
              'customfield_10437': <JIRA CustomFieldOption: value=' ', id='10